In [1]:

# utils.py

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import streamlit as st  
import plotly.express as px

def get_filtered_df(df, selected_dashboards, selected_positions):
    filtered = df.copy()
    if selected_dashboards:
        filtered = filtered[filtered["# Dashboard"].isin(selected_dashboards)]
    if selected_positions:
        filtered = filtered[filtered["Position"].isin(selected_positions)]
    return filtered


def build_donut_chart(lis_data):
    ''' 
    LIS Stands for Leadership Index Score which is a weighted score of critical skills necessary skills and beneficial skills to have a standardised metric to compare all individuals
    EQ assesses Emotional Intelligence which is one of the most important skills that all leaders are assessed in.
    '''
    mean_lis = np.mean(lis_data)
    std_lis = np.std(lis_data)
    std_low = mean_lis - 1.5 * std_lis
    std_high = mean_lis + 1.5 * std_lis

    leaders_meeting = np.sum((lis_data >= std_low) & (lis_data <= std_high))
    leaders_exceeding = np.sum(lis_data > std_high)
    leaders_requiring_training = np.sum(lis_data < std_low)
    
    leaders_summary = {
    "meeting_standard": np.sum((lis_data >= std_low) & (lis_data <= std_high)),
    "exceeding_standard": np.sum(lis_data > std_high),
    "requiring_training": np.sum(lis_data < std_low)
    }

    labels = ['Meeting Minimum Competency', 'Exceeding Expectations', 'Requiring Training']
    values = [leaders_meeting, leaders_exceeding, leaders_requiring_training]
    colors = ['#5c9acc', '#f4a300', '#e63946']

    fig = go.Figure(data=[go.Pie(
        labels=labels,
        values=values,
        hole=0.4,
        marker=dict(colors=colors),
        hovertemplate="%{label}: %{value} leaders (%{percent})",
        textinfo='percent+label'
    )])
    fig.update_layout(
        title="Overall Leadership Competency Levels",
        margin=dict(t=50, b=50, l=50, r=50),
        width=700,
        height=500
    )

    return fig, leaders_summary


def build_histogram(lis_data):
    ''' 
    LIS Stands for Leadership Index Score which is a weighted score of critical skills necessary skills and beneficial skills to have a standardised metric to compare all individuals
    EQ assesses Emotional Intelligence which is one of the most important skills that all leaders are assessed in.
    '''
    mean_lis = np.mean(lis_data)
    std_lis = np.std(lis_data)
    std_low = mean_lis - 1.5 * std_lis
    std_high = mean_lis + 1.5 * std_lis

    below_std = np.sum(lis_data < std_low)
    above_std = np.sum(lis_data > std_high)

    # Summary dictionary
    summary_stats = {
        "mean": mean_lis,
        "std_dev": std_lis,
        "1.5_std_below": std_low,
        "1.5_std_above": std_high,
        "min_value": np.min(lis_data),
        "max_value": np.max(lis_data),
        "count": len(lis_data),
        "below_1.5_std": below_std,
        "above_1.5_std": above_std
    }
    

    fig = go.Figure()
    fig.add_trace(go.Histogram(
        x=lis_data,
        nbinsx=20,
        marker_color='darkblue',
        opacity=0.75,
        name='LIS'
    ))
    fig.add_vline(x=mean_lis, line=dict(color='gold', dash='dash'),
                  annotation_text=f'Mean: {mean_lis:.2f}', annotation_position="top right")
    fig.add_vline(x=std_low, line=dict(color='green', dash='dash'),
                  annotation_text=f'1.5-std below: {std_low:.2f}', annotation_position="top left")
    fig.add_vline(x=std_high, line=dict(color='green', dash='dash'),
                  annotation_text=f'1.5-std above: {std_high:.2f}', annotation_position="top right")
    fig.update_layout(
        title="Leadership Index Score (LIS) Distribution",
        xaxis_title="Leadership Index Score (LIS) Score",
        yaxis_title="Frequency",
        template="plotly_white",
        width=700,
        height=400,
        margin=dict(t=50, b=50, l=50, r=50)
    )
    return fig, summary_stats

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def radar_chart_plotly(dashboard, df, skills_mapping):
    ''' 
    LIS Stands for Leadership Index Score which is a weighted score of critical skills necessary skills and beneficial skills to have a standardized metric to compare all individuals.
    EQ assesses Emotional Intelligence which is one of the most important skills that all leaders are assessed in.
    '''
    mapping = skills_mapping.get(dashboard, {})
    categories = ['Critical Skills', 'Necessary', 'Beneficial Skills']

    avg_scores = {}
    summary_stats = {"dashboard": dashboard, "skill_scores": {}}

    fig = make_subplots(
        rows=1, cols=len(categories),
        specs=[[{'type': 'polar'} for _ in categories]],
        subplot_titles=[f"{dashboard} - {cat}" for cat in categories]
    )

    all_scores = []

    for i, cat in enumerate(categories):
        skills = mapping.get(cat, [])
        if not skills:
            continue

        skill_avg_scores = {
            skill: df.loc[df['# Dashboard'] == dashboard, skill].mean()
            for skill in skills if skill in df.columns
        }
        avg_scores[cat] = list(skill_avg_scores.values())

        # Store per-skill averages in the summary
        summary_stats["skill_scores"][cat] = skill_avg_scores

        scores = np.array(list(skill_avg_scores.values()))
        all_scores.extend(scores)

        # Close the loop for radar visualization
        scores_closed = np.concatenate((scores, [scores[0]]))
        skills_closed = list(skill_avg_scores.keys()) + [list(skill_avg_scores.keys())[0]]

        fig.add_trace(
            go.Scatterpolar(
                r=scores_closed,
                theta=skills_closed,
                fill='toself',
                mode='markers+lines',
                name=cat
            ),
            row=1, col=i+1
        )

        fig.update_polars(
            dict(
                radialaxis=dict(visible=True, range=[0, max(scores_closed) * 1.1]),
                angularaxis=dict(tickfont=dict(size=10))
            ),
            row=1, col=i+1
        )


    fig.update_layout(
        title_text=f"Radar Chart of Average Skill Scores for Dashboard {dashboard}",
        showlegend=False,
        width=350 * len(categories),
        height=500,
        margin=dict(t=100, b=50, l=50, r=50)
    )

    return fig, summary_stats




def build_polar_chart(df):
    ''' 
    LIS Stands for Leadership Index Score which is a weighted score of critical skills necessary skills and beneficial skills to have a standardised metric to compare all individuals
    EQ assesses Emotional Intelligence which is one of the most important skills that all leaders are assessed in.
    '''
    avg_overall = df.groupby("Dashboard Number")["LIS"].mean().reset_index()
    avg_overall_dict=avg_overall.to_dict(orient='records')

    fig = px.bar_polar(
        avg_overall,
        r="LIS",
        theta="Dashboard Number",
        color="Dashboard Number",
        template="plotly_white",
        title="Average Overall Leadership Index Score (LIS) by Dashboard (Polar Bar Chart)",
        color_discrete_sequence=px.colors.qualitative.Bold
    )

    fig.update_layout(
        margin=dict(l=50, r=50, t=100, b=50),
        polar=dict(
            radialaxis=dict(visible=True, range=[0, avg_overall["LIS"].max() * 1.1])
        )
    )
    return fig, avg_overall_dict

def build_box_plot(df):
    ''' 
    LIS Stands for Leadership Index Score which is a weighted score of critical skills necessary skills and beneficial skills to have a standardised metric to compare all individuals
    EQ assesses Emotional Intelligence which is one of the most important skills that all leaders are assessed in.
    '''

    summary_stats = df.groupby("Dashboard Number")["Overall Results"].describe().reset_index()

    fig = px.box(
        df,
        x="Dashboard Number",
        y="Overall Results",
        title="Box Plot of EQ by Dashboard",
        template="plotly_white",
        color="Dashboard Number",
        color_discrete_sequence=px.colors.qualitative.Pastel
    )
    return fig, summary_stats

def get_insights_chart(lis_data, source_code, llm):
    insights_template = PromptTemplate(
        input_variables=["Dataframe", "PlotCode"],
        template="""You are a data analyst helping decision-makers understand data trends.

    Step 1: Analyze the PlotCode and DataFrame to understand what the chart is visualizing. Do NOT include or describe the code itself.

    Step 2: Return your output in **Markdown** using the following structure:

    **Chart Description**
    \n
    A brief paragraph that explains what the chart is visualizing and why it's relevant.


    Important Guidelines:
    - If a KPI, variable, or concept is not clearly defined in the DataFrame or PlotCode, do NOT guess its meaning. It may represent proprietary or internal data.
    - Never fabricate interpretations — only base your insights on what is explicitly observable from the data and plot logic.
    - Be clear, thoughtful, and avoid assumptions.

    Only return the final Markdown-formatted insight (no explanations, no code).

    DataFrame:
    {Dataframe}

    PlotCode:
    {PlotCode}
    """
    )

    #llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
    chain = insights_template | llm | StrOutputParser()
    response = chain.invoke({"Dataframe": lis_data, "PlotCode": source_code})
    return response



# utils.py
import json
import pandas as pd
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA, LLMChain
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

def initialize_pipeline(json_path="result.json"):
    """
    Initializes the pipeline by:
      - Loading the JSON data and converting it to a DataFrame.
      - Creating Document objects for RAG.
      - Setting up the LLM, embeddings, vector store, retriever, and chains.
      - Creating a classifier chain and a unified ask() function.
      
    Returns:
      ask (function): A function that accepts a query string and returns an answer.
    """
    # -----------------------------
    # Load & Prepare JSON Data
    # -----------------------------
    with open(json_path, "r") as f:
        data = json.load(f)

    records = []
    docs = []

    for id_, entry in data.items():
        flat = {"ID": id_}
        flat.update(entry)
        records.append(flat)

        # Create a text version for RAG documents
        text = f"ID: {id_}\n"
        for k, v in entry.items():
            if isinstance(v, list):
                text += f"{k}: {', '.join(map(str, v))}\n"
            else:
                text += f"{k}: {v}\n"
        docs.append(Document(page_content=text))

    df = pd.DataFrame(records)

    # -----------------------------
    # LLM & Embeddings
    # -----------------------------
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")
    embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    # -----------------------------
    # RAG Setup
    # -----------------------------
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    vectorstore = Chroma.from_documents(docs, embedding, collection_name="ldp_docs")
    retriever = vectorstore.as_retriever(search_kwargs={"k": 20, "timeout": 120})

    # Custom prompt for the RAG chain
    context_prompt = PromptTemplate.from_template("""
You are a data assistant helping analyze leadership assessment data.

Use the provided context to answer the question below. The context includes information on individuals, dashboards, and their skills.

Context:
{context}

Question:
{question}

Answer by combining insights across individuals. Focus on patterns, averages, and summaries. If no useful context is found, say "Not enough information."
""")
    llm_chain = LLMChain(llm=llm, prompt=context_prompt)

    # Using StuffDocumentsChain to combine documents into context
    from langchain.chains.combine_documents.stuff import StuffDocumentsChain as CombineStuffDocumentsChain
    stuff_chain = CombineStuffDocumentsChain(
        llm_chain=llm_chain,
        document_variable_name="context"
    )

    rag_chain = RetrievalQA(
        retriever=retriever,
        combine_documents_chain=stuff_chain,
        memory=memory
    )

    # -----------------------------
    # Pandas Agent Setup
    # -----------------------------
    pandas_agent = create_pandas_dataframe_agent(
        llm=llm,
        df=df,
        verbose=True,
        allow_dangerous_code=True
    )

    # -----------------------------
    # Classifier to Route Query
    # -----------------------------
    classifier_prompt = PromptTemplate.from_template("""
You are a smart classifier. Given a user question, decide if it should be handled using:

- "structured" → if it's about rankings, filters, math, comparisons, scores
- "semantic" → if it's about summaries, meaning, types of skills, descriptions

Return ONLY the word "structured" or "semantic".

Question: {query}
""")
    classifier_chain = LLMChain(llm=llm, prompt=classifier_prompt)

    # -----------------------------
    # Unified Ask Function
    # -----------------------------
    def ask(query: str) -> str:
        route = classifier_chain.run({"query": query}).strip().lower()
        print(f"[Routing → {route}]")
        
        if route == "structured":
            return pandas_agent.run(query)
        elif route == "semantic":
            return rag_chain.run(query)
        else:
            return "Sorry, I couldn't confidently classify your question."

    return ask



In [6]:
import inspect
from langchain_google_genai import ChatGoogleGenerativeAI
import os

os.environ['GOOGLE_API_KEY'] = "api"
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

df = pd.read_csv('LDP_summary_anonymized.csv')

filtered = df.copy()
lis_data = filtered['LIS']

source_code = inspect.getsource(build_donut_chart)
test=build_donut_chart(lis_data=lis_data)
response = get_insights_chart(lis_data=a, source_code=source_code, llm=llm)   


In [7]:
from IPython.display import Markdown

Markdown(response)

**Chart Description**

The chart displays the distribution of leadership competency levels among a group of leaders.  It categorizes leaders into three groups: those meeting minimum competency standards, those exceeding expectations, and those requiring additional training. The data shows that the vast majority (68) meet the minimum standards, a smaller number (6) exceed expectations, and a small number (7) require training.  This visualization is relevant for assessing the overall effectiveness and development needs of the leadership team.

In [4]:
import pandas as pd

df = pd.read_csv('../LDP_summary_anonymized.csv')

df

,# Dashboard,ID,Account,Last name,Middle name,First name,Position,Email,Date completed,Completion time,...,Intrapersonal Skills,Accountability,Change Management,Drive,Positive Mindset_manssa,Steady Effort,Key Skills,Necessary Skills,Beneficial Skills,LIS
0,D12: Corporate Communications and Training,LDP0394,LDP100,Finchum,NaN,Donald,CORPORATE CULTURE & TRAINING MANAGER,d.finchum31@anondomain.com,NaN,NaN,...,88,93,83,100,79,86,95.25,92.20,95.2,90.5050
1,D4: Logistics and Site Management,LDP4315,LDP100,Clark,NaN,Susan,SITE MANAGER - CIVIL,s.clark59@anondomain.com,2025-01-14,01:08:55,...,59,68,50,73,35,73,80.00,77.00,61.6,73.3140
2,D6: Procurement and Supply Chain Management,LDP0127,LDP100,Mendoza,NaN,John,PROCUREMENT MANAGER-OFFICE SUPPLY,j.mendoza41@anondomain.com,2025-01-10,01:17:10,...,77,68,80,93,86,64,78.00,81.50,81.0,76.2050
3,D5: Engineering and Technical Management,LDP4802,LDP100,Spegal,NaN,Juan,COMMISSIONING MANAGER,j.spegal48@anondomain.com,2025-01-13,02:24:32,...,77,67,64,97,81,71,81.20,74.50,82.4,75.4210
4,D13: Administrative and Support Services,LDP1313,LDP100,Kingsley,NaN,Ruth,SUPPORT SERVICES MANAGER,r.kingsley18@anondomain.com,2025-01-12,01:37:15,...,84,77,86,100,83,76,89.50,79.60,89.2,81.1940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,D3: Senior Management - Operations and Logistics,LDP3000,LDP 100,Woll,NaN,Deborah,PROJECT MANAGER-CIVIL,d.woll16@anondomain.com,2025-03-25,00:04:48,...,84,80,97,73,93,66,71.00,58.25,70.0,57.0325
77,D3: Senior Management - Operations and Logistics,LDP3019,LDP 100,Shirk,NaN,Robert,PROJECT MANAGER-INDUSTRIAL,r.shirk38@anondomain.com,2025-03-26,01:34:31,...,66,83,73,83,51,54,82.00,81.50,84.8,76.5470
78,D4: Logistics and Site Management,LDP4011,LDP 100,Gomez,NaN,Monique,SITE MANAGER-ELECTRICAL,m.gomez13@anondomain.com,2025-03-26,00:36:47,...,87,72,83,100,91,89,90.60,93.00,90.6,85.6640
79,D5: Engineering and Technical Management,LDP5422,LDP 100,Trowell,NaN,Michael,CONSTRUCTION MANAGER-MEP,m.trowell14@anondomain.com,2025-03-27,01:11:50,...,79,73,71,93,70,91,87.20,81.50,84.0,80.4350


In [5]:
df.columns

Index(['# Dashboard', 'ID', 'Account', 'Last name', 'Middle name',
       'First name', 'Position', 'Email', 'Date completed', 'Completion time',
       'Overall Results',
       'Emotional Identification, Perception, and Expression',
       'Emotional Facilitation of Thought', 'Emotional Understanding',
       'Emotional Management', 'Emotional Self-awareness',
       'Awareness of Strengths and Limitations', 'Comfort with Emotions',
       'Recognition of Other's Emotions', 'Rumination', 'Problem-Solving',
       'Positive Mindset', 'Emotional Reflection', 'Emotional Integration',
       'Conflict-Resolution Knowledge', 'Empathy', 'Social Insight',
       'Self-Control', 'Resilience/Hardiness', 'Coping Skills',
       'Self-Motivation', 'Striving', 'Emotional Selectivity',
       'Adaptable Social Skills', 'Conflict-Resolution Behavior',
       'Account_manssa', 'Last name_manssa', 'Middle name_manssa',
       'First name_manssa', 'Date completed_manssa', 'Completion time_manssa',
  

In [19]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def radar_chart_plotly(dashboard, df, skills_mapping):
    ''' 
    LIS Stands for Leadership Index Score which is a weighted score of critical skills necessary skills and beneficial skills to have a standardized metric to compare all individuals.
    EQ assesses Emotional Intelligence which is one of the most important skills that all leaders are assessed in.
    '''
    mapping = skills_mapping.get(dashboard, {})
    categories = ['Critical Skills', 'Necessary', 'Beneficial Skills']

    avg_scores = {}
    summary_stats = {"dashboard": dashboard, "skill_scores": {}}

    fig = make_subplots(
        rows=1, cols=len(categories),
        specs=[[{'type': 'polar'} for _ in categories]],
        subplot_titles=[f"{dashboard} - {cat}" for cat in categories]
    )

    all_scores = []

    for i, cat in enumerate(categories):
        skills = mapping.get(cat, [])
        if not skills:
            continue

        skill_avg_scores = {
            skill: df.loc[df['# Dashboard'] == dashboard, skill].mean()
            for skill in skills if skill in df.columns
        }
        avg_scores[cat] = list(skill_avg_scores.values())

        # Store per-skill averages in the summary
        summary_stats["skill_scores"][cat] = skill_avg_scores

        scores = np.array(list(skill_avg_scores.values()))
        all_scores.extend(scores)

        # Close the loop for radar visualization
        scores_closed = np.concatenate((scores, [scores[0]]))
        skills_closed = list(skill_avg_scores.keys()) + [list(skill_avg_scores.keys())[0]]

        fig.add_trace(
            go.Scatterpolar(
                r=scores_closed,
                theta=skills_closed,
                fill='toself',
                mode='markers+lines',
                name=cat
            ),
            row=1, col=i+1
        )

        fig.update_polars(
            dict(
                radialaxis=dict(visible=True, range=[0, max(scores_closed) * 1.1]),
                angularaxis=dict(tickfont=dict(size=10))
            ),
            row=1, col=i+1
        )


    fig.update_layout(
        title_text=f"Radar Chart of Average Skill Scores for Dashboard {dashboard}",
        showlegend=False,
        width=350 * len(categories),
        height=500,
        margin=dict(t=100, b=50, l=50, r=50)
    )

    return fig, summary_stats


a, b = radar_chart_plotly("D12: Corporate Communications and Training", df, skills_mapping)
b


{'dashboard': 'D12: Corporate Communications and Training',
 'skill_scores': {'Critical Skills': {'Communication Skills': 75.6,
   'Training & Onboarding': 84.8,
   'Coaching & Mentoring': 83.8,
   'Goal Setting': 90.4},
  'Necessary': {'Organization Skills': 79.0,
   'Measuring & Evaluating Results': 90.0,
   'Conflict Management': 71.6,
   'Listening Skills': 65.2,
   'Building Effective Teams': 84.4},
  'Beneficial Skills': {'Social Insight': 100.0,
   'Supportiveness': 84.0,
   'Charisma': 84.8,
   'Emotional Self-awareness': 59.6,
   'Change Management': 74.4}}}

In [20]:
a

In [8]:
from langchain_core.prompts import PromptTemplate

# Define the prompt template exactly as in your get_insights_chart function
insights_template = PromptTemplate(
    input_variables=["Dataframe", "PlotCode"],
    template="""You are a data analyst helping decision-makers understand data trends.

Step 1: Analyze the PlotCode and DataFrame to understand what the chart is visualizing. Do NOT include or describe the code itself.

Step 2: Return your output in **Markdown** using the following structure:

**Chart Description**

A brief paragraph that explains what the chart is visualizing and why it's relevant.

Important Guidelines:
- If a KPI, variable, or concept is not clearly defined in the DataFrame or PlotCode, do NOT guess its meaning. It may represent proprietary or internal data.
- Never fabricate interpretations — only base your insights on what is explicitly observable from the data and plot logic.
- Be clear, thoughtful, and avoid assumptions.

Only return the final Markdown-formatted insight (no explanations, no code).

DataFrame:
{Dataframe}

PlotCode:
{PlotCode}"""
)

# Now format the prompt with your inputs
formatted_prompt = insights_template.format(Dataframe=test, PlotCode=source_code)
print("Final prompt passed to LLM:\n", formatted_prompt)


Final prompt passed to LLM:
 You are a data analyst helping decision-makers understand data trends.

Step 1: Analyze the PlotCode and DataFrame to understand what the chart is visualizing. Do NOT include or describe the code itself.

Step 2: Return your output in **Markdown** using the following structure:

**Chart Description**

A brief paragraph that explains what the chart is visualizing and why it's relevant.

Important Guidelines:
- If a KPI, variable, or concept is not clearly defined in the DataFrame or PlotCode, do NOT guess its meaning. It may represent proprietary or internal data.
- Never fabricate interpretations — only base your insights on what is explicitly observable from the data and plot logic.
- Be clear, thoughtful, and avoid assumptions.

Only return the final Markdown-formatted insight (no explanations, no code).

DataFrame:
{'meeting_standard': 68, 'exceeding_standard': 6, 'requiring_training': 7}

PlotCode:
def build_donut_chart(lis_data):
    ''' 
    LIS Stand